In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from hyperopt import hp
from ray.tune.search.bayesopt import BayesOptSearch
from ray.tune.schedulers import ASHAScheduler
from functools import partial
from ray.experimental.tqdm_ray import tqdm
from ray.util.accelerators import NVIDIA_TESLA_V100

import torch
from ray import train, tune
from ray.tune.search.optuna import OptunaSearch
from ray import train

from sklearn.model_selection import TimeSeriesSplit
import math
import matplotlib.pyplot as plt
import talib
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("gpu")
else:
    device = torch.device('cpu')
print(torch.__version__)
print('CUDA available:', torch.cuda.is_available())
print('CUDA version:', torch.version.cuda)
print('cuDNN version:', torch.backends.cudnn.version())

2.1.2
CUDA available: False
CUDA version: None
cuDNN version: None


In [3]:
def add_technical_indicators(spread, timeperiod=22):

    # MACD
    macd, macdsignal, macdhist = talib.MACD(spread, fastperiod=12, slowperiod=26, signalperiod=9)

    rsi = talib.RSI(spread, timeperiod=14)

    # CMO
    cmo = talib.CMO(spread, timeperiod=timeperiod)

    # MOM
    mom = talib.MOM(spread, timeperiod=timeperiod)

    # Bollinger Bands
    upperband, middleband, lowerband = talib.BBANDS(spread, timeperiod=timeperiod, nbdevup=2, nbdevdn=2, matype=0)

    # SMA
    sma = talib.SMA(spread, timeperiod=timeperiod)

    # Combine all indicators into a DataFrame
    indicators = pd.DataFrame({
        'MACD': macd,
        'MACD_signal': macdsignal,
        'MACD_hist': macdhist,
        'RSI': rsi,
        'CMO': cmo,
        'MOM': mom,
        'Upper_BB': upperband,
        'Middle_BB': middleband,
        'Lower_BB': lowerband,
        'SMA': sma
    })
    return indicators


In [4]:
# Load and preprocess data
data = pd.read_csv("Technology_Firm_Stock_Price.csv")
data = data.sort_values(by="Date")
selected_pairs = [('SWKS', 'QRVO'), ('AMAT', 'NXPI'), ('ANET', 'PANW'), ('CDNS', 'PANW')]
spreads_with_indicators = {}
for stock1, stock2 in selected_pairs:
    spread = data[stock1] - data[stock2]
    indicators = add_technical_indicators(spread)
    indicators['Spread'] = spread
    spreads_with_indicators[(stock1, stock2)] = indicators.dropna()  # Drop rows with NaN 22 column (because of our timeperiod is 22 day for calculate these indicators thus we dropped first 22 column

spreads_with_indicators


{('SWKS',
  'QRVO'):           MACD  MACD_signal  MACD_hist        RSI        CMO       MOM  \
 33    2.030602     1.107445   0.923157  73.712986  37.256446  7.412430   
 34    2.264091     1.338774   0.925317  74.970416  39.211667  8.581909   
 35    2.313134     1.533646   0.779488  68.137934  30.893807  7.610367   
 36    1.931359     1.613189   0.318171  50.189722   6.584774  2.025627   
 37    1.472331     1.585017  -0.112686  45.654763  -0.213434  0.463966   
 ...        ...          ...        ...        ...        ...       ...   
 1001 -0.036478     0.001545  -0.038023  47.853326  -3.562012 -0.310005   
 1002 -0.068166    -0.012397  -0.055769  47.236833  -4.324748 -0.789993   
 1003 -0.118541    -0.033626  -0.084915  45.349182  -6.645434 -1.709999   
 1004 -0.161444    -0.059189  -0.102255  44.997068  -7.074799 -1.520004   
 1005 -0.187634    -0.084878  -0.102756  45.528411  -6.476312 -1.459999   
 
       Upper_BB  Middle_BB  Lower_BB       SMA    Spread  
 33    7.503425   1

In [5]:
def rolling_window(data):
    time_step = 22

    X, y = [], []

    for i in range(len(data) - time_step):
        X.append(data.iloc[i : (i+time_step)])
        y.append(data.iloc[i+time_step])

    X = np.array(X)
    y = np.array(y)

    print(X.shape)
    print(y.shape)

    return X, y

In [6]:
spreads_with_indicators[('SWKS','QRVO')].head()

,MACD,MACD_signal,MACD_hist,RSI,CMO,MOM,Upper_BB,Middle_BB,Lower_BB,SMA,Spread
33,2.030602,1.107445,0.923157,73.712986,37.256446,7.412430,7.503425,1.239307,-5.024811,1.239307,7.912239
34,2.264091,1.338774,0.925317,74.970416,39.211667,8.581909,8.574625,1.629394,-5.315838,1.629394,8.608337
35,2.313134,1.533646,0.779488,68.137934,30.893807,7.610367,9.241053,1.975320,-5.290414,1.975320,7.253357
36,1.931359,1.613189,0.318171,50.189722,6.584774,2.025627,9.296648,2.067394,-5.161861,2.067394,2.316322
37,1.472331,1.585017,-0.112686,45.654763,-0.213434,0.463966,9.297611,2.088483,-5.120646,2.088483,0.587555


In [8]:
scaled_spreads_with_indicators = {}  # Dictionary to hold scaled data

for key, dataframe in spreads_with_indicators.items():
    dataframe = dataframe.replace([np.inf], 1)
    dataframe = dataframe.replace([-np.inf], 0)

    #dataframe = dataframe.dropna()
    spreads_with_indicators[key] = dataframe

for key, dataframe in spreads_with_indicators.items():

    scaled_dataframe = pd.DataFrame()  # DataFrame to hold scaled values for the current key
    scaler = MinMaxScaler(feature_range=(0, 1))  # Initialize the scaler

    for column in dataframe.columns:
        # Scale each column and add it to the scaled_dataframe
        scaled_values = scaler.fit_transform(dataframe[column].values.reshape(-1, 1))
        scaled_dataframe[column] = scaled_values.flatten()  # Flatten the array to 1D and set it as a column

    # Add the scaled DataFrame to the dictionary
    scaled_spreads_with_indicators[key] = scaled_dataframe

# Now scaled_spreads_with_indicators contains the scaled data.
print(scaled_spreads_with_indicators)


{('SWKS', 'QRVO'):          MACD  MACD_signal  MACD_hist       RSI       CMO       MOM  Upper_BB  \
0    0.583049     0.546882   0.560607  0.876858  0.854227  0.524526  0.711123   
1    0.599417     0.564780   0.561000  0.896479  0.872540  0.540631  0.738404   
2    0.602855     0.579858   0.534516  0.789862  0.794632  0.527252  0.755377   
3    0.576091     0.586012   0.450739  0.509789  0.566945  0.450345  0.756793   
4    0.543912     0.583832   0.372493  0.439024  0.503271  0.428840  0.756817   
..        ...          ...        ...       ...       ...       ...       ...   
968  0.438139     0.461318   0.386052  0.473331  0.471907  0.418181  0.580229   
969  0.435918     0.460239   0.382830  0.463711  0.464762  0.411571  0.580040   
970  0.432386     0.458597   0.377537  0.434255  0.443026  0.398902  0.578656   
971  0.429379     0.456619   0.374388  0.428761  0.439005  0.401519  0.577778   
972  0.427543     0.454631   0.374297  0.437052  0.444610  0.402345  0.576469   

     Mid

In [4]:
class RollingWindowDataset(Dataset):
    def __init__(self, X, y, device="gpu"):
        self.X = torch.tensor(X, dtype=torch.float, device=device)
        self.y = torch.tensor(y, dtype=torch.float, device=device)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx].clone().detach().to(torch.float), self.y[idx].clone().detach().to(torch.float)


In [11]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()

        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        self.lstm = nn.LSTM(input_size = input_dim, hidden_size = hidden_dim, num_layers = layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initializing hidden state
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(device)

        # Initialize cell state
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(device)

        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :])

        return out

In [ ]:
train_data = RollingWindowDataset(X_train, y_train, device)
test_data = RollingWindowDataset(X_test, y_test, device)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = LSTMModel(input_dim=X_train.shape[2], hidden_dim=22, layer_dim=1, output_dim=y_train.shape[1]).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.MSELoss()

In [17]:
class LSTM_Trainer:
    def __init__(self, train_data, test_data, device, optimizer, criterion):
        self.train_data = train_data
        self.test_data = test_data
        self.device = device
        self.optimizer = optimizer
        self.criterion =  criterion
        self.model = LSTMModel(input_dim=1, hidden_dim=22, layer_dim=1, output_dim=1).to(self.device)

    def train_lstm(self, config, checkpoint_dir=None):

        batch_size = int(round(config["batch_size"]))
        epochs = int(round(config["epochs"]))
        hidden_dim = int(round(config["hidden_dim"]))
        layer_dim = int(round(config["layer_dim"]))
        learning_rate = config["learning_rate"]

        # Create the model
        self.model = LSTMModel(input_dim=1, hidden_dim=hidden_dim, layer_dim=layer_dim, output_dim=1).to(self.device)

        # Define loss and optimizer
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

        # Define data loader (you may need to adapt this based on your data format)
        train_loader = DataLoader(self.train_data, batch_size=batch_size, shuffle=False)

        # Training loop
        for epoch in range(epochs):
            print('epochs {}/{}'.format(epoch+1,epochs))
            running_loss = .0
            self.model.train()


            for batch_idx, (data, target) in enumerate(train_loader):
                if isinstance(data, np.ndarray):
                    data = torch.from_numpy(data).float()
                if isinstance(target, np.ndarray):
                    target = torch.from_numpy(target).float()

                data, target = data.to(self.device), target.to(self.device)

                self.optimizer.zero_grad()
                predictions = self.model(data)
                loss = self.criterion(predictions, target)
                loss.backward()
                self.optimizer.step()

                running_loss += loss.item()


            train_loss = running_loss/len(train_loader)
            print(train_loss)
            train.report({'mse': train_loss})


    def Test(self, config, test_data):
        batch_size = int(round(config["batch_size"]))
        self.test_data = test_data

        all_predictions = []
        test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

        running_loss = .0

        self.model.eval()

        with torch.no_grad():

            for batch_idx, (data, target) in enumerate(test_loader):
                if isinstance(data, np.ndarray):
                    data = torch.from_numpy(data).float()
                if isinstance(target, np.ndarray):
                    target = torch.from_numpy(target).float()

                data, target = data.to(self.device), target.to(self.device)

                self.optimizer.zero_grad()
                preds = self.model(data)
                loss = self.criterion(preds,target)
                running_loss += loss.item()

                all_predictions.extend(preds.cpu().numpy())


            test_loss = running_loss/len(test_loader)

            print(f'test_loss {test_loss}')

        all_predictions = np.array(all_predictions)

        return all_predictions


    def get_current_model(self):
        if self.model is not None:
            return self.model
        else:
            raise ValueError("Model has not been trained yet.")


In [ ]:
def train_model_with_config(config, train_data, test_data, device, optimizer, criterion):
    trainer = LSTM_Trainer(train_data, test_data, device, optimizer, criterion)
    return trainer.train_lstm(config)

In [ ]:
def tuning(train_data, test_data, device, optimizer, criterion):
    ray.shutdown()
    ray.init(num_cpus=8, num_gpus=1)

    # Define hyperparameter configuration
    config = {
        "learning_rate": tune.choice([1e-4, 1e-3, 1e-2]),
        "batch_size": tune.choice([32, 64, 128]),
        "epochs": tune.choice([50, 100, 150]),
        "hidden_dim": tune.choice([22, 35, 50, 75]),
        "layer_dim": tune.choice([1, 2, 3])
    }

    # Define scheduler and reporter
    scheduler = ASHAScheduler(
        metric="mse",
        mode="min",
        grace_period=15,
        reduction_factor=2
    )
    reporter = CLIReporter(metric_columns=["mse", "training_iteration"])

    # Optuna search algorithm
    optuna_search = OptunaSearch(metric="mse", mode="min")

    # Partial function for training with LSTM and given data
    #train_lstm_with_data = partial(trainer.train_lstm, device=trainer.device)

    trainable_with_cpu_gpu = tune.with_resources(lambda config: train_model_with_config(config, train_data, test_data, device, optimizer, criterion), {"cpu": 8, "gpu": 1})


    # Start the tuning process
    analysis = tune.run(
        trainable_with_cpu_gpu,
        # resources_per_trial={"cpu": 8, "gpu": 1},
        config=config,
        num_samples=50,
        scheduler=scheduler,
        search_alg=optuna_search,
        progress_reporter=None
    )

    # Get the best hyperparameters
    best_config = analysis.get_best_config(metric="mse", mode="min")
    best_mse = analysis.get_best_trial(metric="mse", mode="min").last_result["mse"]

    print("Best config: ", best_config)
    print(f"Best MSE: {best_mse}")

    return best_config

In [ ]:
score = []
checked_compt = set()


for ticker in tqdm(data.columns, desc="Processing tickers"):

    train_data = RollingWindowDataset(X_train, y_train, device)
    test_data = RollingWindowDataset(X_test, y_test, device)

    best_config = tuning(train_data, test_data, device, optimizer, criterion)

    trainer = LSTM_Trainer(train_data, test_data, device, optimizer, criterion)
    trainer.train_lstm(best_config)
    preds = trainer.Test(best_config, test_data)

    mape = mean_absolute_percentage_error(y_test, preds) * 100
    score.append((mape, ticker))

    for pair in data.columns:
        comp_data_X = company_dict[pair]["X"]
        comp_data_y = company_dict[pair]["y"]
        X_train, X_test, y_train, y_test = train_test_split(comp_data_X, comp_data_y, test_size=0.2, shuffle=False)

        test_data = RollingWindowDataset(X_test, y_test, device)

        preds = trainer.Test(best_config, test_data)

        pair_mape = mean_absolute_percentage_error(y_test, preds) * 100

        score.append((mape, ticker , pair, pair_mape))


In [ ]:
score

In [20]:
# Example usage with one stock pair
pair = selected_pairs[0]  # Example: first pair in the list
spread_data = spreads_with_indicators[pair]
x, y = rolling_window(spread_data)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)
#print(x_train.shape)

# Train the LSTM model
model = train_lstm(pair, x_train, y_train, batch_size, epochs, hidden_dim, num_layers, learning_rate)
# Testing function
def test_lstm(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    predictions = []
    actuals = []
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            predictions.extend(output.numpy())  # Store predictions
            actuals.extend(target.numpy())  # Store actual values
    return np.array(predictions), np.array(actuals)

# MAPE calculation
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_mask = y_true != 0
    return np.mean(np.abs((y_true[non_zero_mask] - y_pred[non_zero_mask]) / y_true[non_zero_mask])) * 100

# Define fixed hyperparameters
batch_size = 32
epochs = 10
hidden_dim = 128
num_layers = 2
learning_rate = 0.001

# Preprocess and prepare data
pair = selected_pairs[0]  # Example: first pair in the list
spread_data = data['SWKS'] - data['QRVO']  # Example spread calculation
x, y = rolling_window(spread_data)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)

# Scaling data (make sure to scale x_train and x_test appropriately)
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train.reshape(-1, x_train.shape[-1])).reshape(x_train.shape)
x_test_scaled = scaler.transform(x_test.reshape(-1, x_test.shape[-1])).reshape(x_test.shape)



# Testing
test_loader = DataLoader(RollingWindowDataset(x_test_scaled, y_test), batch_size=batch_size, shuffle=False)
predictions, actuals = test_lstm(model, test_loader)

# Calculate and print MAPE
mape = mean_absolute_percentage_error(actuals[:, 2], predictions[:, 2])  # Assuming you want the 3rd column
print(f"MAPE: {mape:.2f}%")

(951, 22, 11)
(951, 11)
Pair: ('SWKS', 'QRVO'), Epoch 1/10, Loss: 319.219394
Pair: ('SWKS', 'QRVO'), Epoch 2/10, Loss: 264.257516
Pair: ('SWKS', 'QRVO'), Epoch 3/10, Loss: 233.387033
Pair: ('SWKS', 'QRVO'), Epoch 4/10, Loss: 210.152650
Pair: ('SWKS', 'QRVO'), Epoch 5/10, Loss: 191.006456
Pair: ('SWKS', 'QRVO'), Epoch 6/10, Loss: 175.112144
Pair: ('SWKS', 'QRVO'), Epoch 7/10, Loss: 161.713285
Pair: ('SWKS', 'QRVO'), Epoch 8/10, Loss: 150.318541
Pair: ('SWKS', 'QRVO'), Epoch 9/10, Loss: 140.438893
Pair: ('SWKS', 'QRVO'), Epoch 10/10, Loss: 132.159720
(984, 22)
(984,)


RuntimeError: For unbatched 2-D input, hx and cx should also be 2-D but got (3-D, 3-D) tensors

In [ ]:
# Calculate and print MAPE
mape = mean_absolute_percentage_error(actuals[:, 2], predictions[:, 2])  # Assuming you want the 3rd column
print(f"MAPE: {mape:.2f}%")

# Visualization of the 3rd column of actuals vs predictions
plt.figure(figsize=(12, 6))
plt.plot(actuals[:, 2], label='Actual Values (3rd Column)', color='blue')  # Plotting the 3rd column of actuals
plt.plot(predictions[:, 2], label='Predicted Values (3rd Column)', color='red')  # Plotting the 3rd column of predictions
plt.title('LSTM Model Predictions vs Actual Values (3rd Column)')
plt.xlabel('Time Steps')
plt.ylabel('Values')
plt.legend()
plt.show()

In [ ]:
# Create a MinMaxScaler instance
scaler = MinMaxScaler(feature_range=(-1, 1))

selected_data['1. spread'] = scaler.fit_transform(selected_data['1. spread'].values.reshape(-1,1))
scaled_data = selected_data
scaled_data = pd.DataFrame(scaled_data, columns=selected_data.columns)


scaled_data

In [ ]:
def load_data(stock, look_back):
    data_raw = stock.values # convert to numpy array
    data = []

    # create all possible sequences of length look_back
    for index in range(len(data_raw) - look_back):
        data.append(data_raw[index: index + look_back])

    data = np.array(data);
    test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);

    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]

    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]

    return [x_train, y_train, x_test, y_test]

look_back = 60 # choose sequence length
x_train, y_train, x_test, y_test = load_data(scaled_data, look_back)
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ',y_train.shape)
print('x_test.shape = ',x_test.shape)
print('y_test.shape = ',y_test.shape)

In [ ]:
# make training and test sets in torch
x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train = torch.from_numpy(y_train).type(torch.Tensor)
y_test = torch.from_numpy(y_test).type(torch.Tensor)

In [ ]:
y_train.size(),x_train.size()

In [ ]:
class RollingWindowDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.float), torch.tensor(self.y[idx], dtype=torch.float)
# Define LSTM model
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :])
        return out
# Parameters
look_back = 60
hidden_dim = 32
num_layers = 2
output_dim = 1
num_epochs = 100
loss_fn = torch.nn.MSELoss()

In [ ]:
class RollingWindowDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.float), torch.tensor(self.y[idx], dtype=torch.float)

In [ ]:
# Build model
#####################
input_dim = 1
hidden_dim = 32
num_layers = 2
output_dim = 1


# Here we define our model as a class
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.num_layers = num_layers

        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()

        # Initialize cell state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()

        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Index hidden state of last time step
        # out.size() --> 100, 32, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states!
        out = self.fc(out[:, -1, :])
        # out.size() --> 100, 10
        return out

model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)

loss_fn = torch.nn.MSELoss()

optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
print(model)
print(len(list(model.parameters())))
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

In [ ]:
# Train model
#####################
num_epochs = 100
hist = np.zeros(num_epochs)

# Number of steps to unroll
seq_dim =look_back-1

for t in range(num_epochs):
    # Initialise hidden state
    # Don't do this if you want your LSTM to be stateful
    #model.hidden = model.init_hidden()

    # Forward pass
    y_train_pred = model(x_train)

    loss = loss_fn(y_train_pred, y_train)
    if t % 10 == 0 and t !=0:
        print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()

    # Zero out gradient, else they will accumulate between epochs
    optimiser.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    optimiser.step()

In [ ]:
plt.plot(hist, label="Training loss")
plt.legend()
plt.show()

In [ ]:
np.shape(y_train_pred)

In [ ]:
# make predictions
y_test_pred = model(x_test)

# invert predictions
y_train_pred = scaler.inverse_transform(y_train_pred.detach().numpy())
y_train = scaler.inverse_transform(y_train.detach().numpy())
y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())
y_test = scaler.inverse_transform(y_test.detach().numpy())

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(y_train[:,0], y_train_pred[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test[:,0], y_test_pred[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# Visualising the results
figure, axes = plt.subplots(figsize=(15, 6))
axes.xaxis_date()

axes.plot(scaled_data[len(scaled_data)-len(y_test):].index, y_test, color = 'red', label = 'Actual Spread')
axes.plot(scaled_data[len(scaled_data)-len(y_test):].index, y_test_pred, color = 'blue', label = 'Predicted Spread')
#axes.xticks(np.arange(0,394,50))
plt.title('Spread Prediction')
plt.xlabel('Time')
plt.ylabel('Spread Price')
plt.legend()
plt.savefig('spread.png')
plt.show()